In [89]:
import json
import pickle
import random
from collections import defaultdict
from datetime import datetime
from itertools import chain
from math import log
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from matplotlib import pyplot
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold
from statsmodels.formula import api as formula_api

%matplotlib inline
pandas.set_option('display.max_columns', 500)

In [90]:
# Define cross-validation and evaluation strategies.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = 'accuracy'

In [91]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        result[f'level_{hero["id"]}'] += sign * hero['level']
        result[f'color_{hero["id"]}'] += sign * hero['color']
        result[f'star_{hero["id"]}'] += sign * hero['star']


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'win': battle['win'], **result}

In [92]:
# Load the logs into a data frame.

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines]).fillna(value=0)
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
battles.head()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9,win
0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False


In [93]:
battles.describe()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9
count,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.00000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000,306.000000
mean,-0.555556,3.356209,1.271242,-1.316993,3.643791,-0.316993,-0.732026,0.098039,-0.316993,0.281046,-0.124183,1.839869,-1.310458,-0.156863,-0.143791,-0.055556,1.532680,-0.382353,-0.245098,-0.091503,-0.098039,-0.127451,-0.160131,0.022876,-0.326797,-0.098039,-0.186275,0.009804,-0.153595,-0.022876,0.143791,-0.199346,-2.562092,-0.091503,0.006536,-0.045752,-0.741830,-0.326797,-0.905229,-0.287582,-3.937908,28.107843,10.133987,-8.983660,28.666667,-2.372549,-5.816993,0.882353,-2.575163,3.581699,-0.993464,13.555556,-10.872549,-1.228758,-1.169935,-0.392157,12.281046,-3.049020,-1.892157,-0.754902,-0.738562,-0.928105,-1.267974,0.179739,-2.627451,-0.888889,-1.437908,0.173203,-1.156863,-0.163399,1.192810,-1.624183,-20.088235,-0.784314,0.006536,-0.398693,-5.424837,-2.258170,-7.310458,-1.493464,-0.186275,1.905229,0.571895,-0.539216,1.620915,-0.114379,-0.179739,0.04902,-0.127451,0.140523,-0.035948,0.937908,-0.467320,-0.062092,-0.045752,-0.032680,0.091503,-0.124183,-0.101307,-0.029412,-0.026144,-0.039216,-0.101307,0.006536,-0.127451,-0.052288,-0.068627,-0.003268,-0.042484,-0.009804,0.065359,-0.058824,-0.977124,-0.026144,0.013072,-0.019608,-0.232026,0.189542,-0.343137,-0.166667
std,2.658457,3.733839,3.292558,3.709205,3.832635,1.906204,5.085507,0.761919,1.442052,3.983503,1.314835,3.544173,3.112906,1.255115,0.960999,0.595215,4.134689,1.625882,1.350917,1.292396,0.855184,1.101459,1.132462,0.400163,1.498984,0.963360,1.278296,1.847436,1.233124,0.400163,1.452723,1.160610,4.342440,0.828663,0.080713,0.570762,2.522427,4.613352,2.620381,1.239636,21.078294,30.510490,26.377680,29.965385,30.097111,15.471553,41.066350,6.857272,11.569430,32.412326,10.261816,28.376598,

In [94]:
# Split the dataframe.

x = battles.drop(['win'], axis=1)
y = battles['win']

In [95]:
# Train, adjust hyper-parameters and evaluate.

numpy.random.seed(42)

classifier = RandomForestClassifier(class_weight='balanced', n_jobs=5, random_state=42)
param_grid = {
    'n_estimators': [43, 44, 45, 46, 47, 48],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 3],
    'max_features': [None, 'auto', 'log2'],
}

grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_jobs=1).fit(x, y)
best_estimator = grid_search.best_estimator_

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {best_estimator.classes_}')

scores = cross_val_score(best_estimator, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.8006535947712419
Best params: {'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 47}
Classes: [False  True]
CV score: 0.8010946589106294 (std: 0.046938872702724156)


In [96]:
# Re-train the best model on the entire dataset.

best_estimator.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=47, n_jobs=5,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [97]:
# Save the model.

print(f'''
"""
Arena battle prediction model.
Auto-generated on {datetime.now().replace(microsecond=0)}.
X shape: {x.shape}.
Score: {scores.mean():.4f} (std: {scores.std():.4f}).
"""

import pickle

from {best_estimator.__class__.__module__} import {best_estimator.__class__.__qualname__}

feature_names = {list(x.columns)}
model: {best_estimator.__class__.__qualname__} = pickle.loads({pickle.dumps(best_estimator)})
'''.strip(), file=open('model.py', 'wt'))

In [98]:
# Display the feature importances.

pandas.DataFrame({'Feature': x.columns, 'Importance': best_estimator.feature_importances_}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=10)

,Importance
Feature,
level_2,0.063833
level_7,0.053172
level_15,0.038080
level_25,0.037047
level_10,0.033360
star_9,0.032874
color_15,0.029662
color_7,0.029230
level_4,0.028196


In [99]:
# Display prediction examples.

result = pandas.concat((
    pandas.Series(best_estimator.predict(x), index=battles.index, name='win_predicted'),
    pandas.Series(best_estimator.predict_proba(x)[:, 1], index=battles.index, name='win_probability'),
    battles,
), axis=1)
result['win_probability'] = result['win_probability'].apply('{:.2f}'.format)
result.head()

,win_predicted,win_probability,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9,win
0,False,0.06,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,False,0.02,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,False,0.04,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,False,0.00,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,False,0.11,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
